In [ ]:
#<img src="Figs/GEOS_Logo.pdf" width="500" />

# `Seasonal Strain`**: <font color=blue>"make_principal_strain.ipynb"</font>
#### Nov 24, 2021  <font color=red>(v. working)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)

> input files: **`average_strain_MATLAB.out`** \
> output files: **`principal_ext.out`** & **`principal_con.out`** & **`dilatation.out`**

0. This code is a part of OOOOO process.
1. It will calculate principal strains (psvelo -Sx) and dilatational strain (xyz)


In [1]:
import numpy as np
import pandas as pd

In [2]:
input_file="average_strain_MATLAB.out"
output_file1="principal_ext.out"
output_file2="principal_con.out"
output_file3="dilatation.out"


data=np.loadtxt(input_file)
long = data[:,1]
lat = data[:,2]
exx = data[:,3]
eyy = data[:,4]
exy = data[:,5]

num_strain = len(exx)

### Calculate Principal strains and save in a GMT format
> The GMT module **psvelo** with -Sx flag requires a input file with a format of  \
> **`lon`, `lat`, `ext`, `contr`, `angle(e22,north)`** 

In [3]:
BASIS_for_extensional = np.zeros((num_strain,5))
BASIS_for_contractional = np.zeros((num_strain,5))

for i in range(num_strain):
    tau = np.array([[exx[i],exy[i]],[exy[i],eyy[i]]])
    Lam, N = np.linalg.eig(tau) #Lam = eigenvalues; N = eigenvectors (columns)
    taur = N.transpose() @ tau @ N  #Principal axes    
    
    # Set the larger principal axis is e11
    t1=taur[0,0]
    t2=taur[1,1]
    N1=N[:,0]
    N2=N[:,1]
    if t2 > t1:
        taur[0,0] = t2
        taur[1,1] = t1
        N = np.array([N2,N1])
        
    # Take the two components of the contractional eigenvector
    x = N[0,1]  
    y = N[1,1]
    
    # Angle from the north
    if x > 0 and y > 0:
        angle = np.arcsin(x)*180/np.pi
    elif x > 0 and y < 0:
        angle = 180 - np.arcsin(x)*180/np.pi
    elif x == 0 and y > 0:
        angle = 0
    elif x == 0 and y < 0:
        angle = 180
    elif x < 0 and y < 0:
        angle = 180 + np.arcsin(np.abs(x))*180/np.pi
    elif x < 0 and y > 0:
        angle = 360 - np.arcsin(np.abs(x))*180/np.pi
    elif x > 0 and y == 0:
        angle = 90
    elif x < 0 and y == 0:
        angle = 270
    else:
        print("ERROR! Stop this algorithm.")
        break
        
        
    BASIS_for_extensional[i,0] = long[i]
    BASIS_for_extensional[i,1] = lat[i]
    BASIS_for_extensional[i,4] = angle
    
    
    BASIS_for_contractional[i,0] = long[i]
    BASIS_for_contractional[i,1] = lat[i]
    BASIS_for_contractional[i,4] = angle
    
    
    if taur[0,0] >= 0 and taur[1,1] >= 0: #e11 and e22 are both extentional (+ve)
        BASIS_for_extensional[i,2] = taur[0,0]
        BASIS_for_extensional[i,3] = taur[1,1]
        
    elif taur[0,0] >= 0 and taur[1,1] < 0: #e11 +ve ; e22 -ve
        BASIS_for_extensional[i,2] = taur[0,0]
        BASIS_for_contractional[i,3] = taur[1,1]
    
    elif taur[0,0] < 0 and taur[1,1] < 0: #e11 and e22 are both contractional (-ve)
        BASIS_for_contractional[i,2] = taur[0,0]
        BASIS_for_contractional[i,3] = taur[1,1]
    else:
        print("ERROR! Stop this algorithm.")
        break
        


### Calculate dilatational strains and save in a xyz format
> **dilatation is invariant** : exx+eyy = e11+e22

In [4]:
BASIS_for_dilatation = data[:,1:4] 
#BE CAREFUL: the third column is exx at this moment
dil = exx + eyy
BASIS_for_dilatation[:,2]=dil

### SAVE THE RESULTS

In [5]:
np.savetxt(output_file1, BASIS_for_extensional, fmt='%g', delimiter=' ')
np.savetxt(output_file2, BASIS_for_contractional, fmt='%g', delimiter=' ')
np.savetxt(output_file3, BASIS_for_dilatation, fmt='%g', delimiter=' ')